In [ ]:
!pip install ytmusicapi
!ytmusicapi oauth


In [1]:
import json
import random
import convert
from ytmusicapi import YTMusic
import pandas as pd

with open("YourLibrary.json", encoding="utf-8") as f:
    library = json.load(f)

with open("Playlist1.json", encoding="utf-8") as f:
    playlists = json.load(f)['playlists']

jsons = ["Streaming_History_Audio_2018-2020_0.json",
"Streaming_History_Audio_2020-2022_1.json",
"Streaming_History_Audio_2022-2023_2.json",
"Streaming_History_Audio_2023-2024_3.json"]
full_streaming_history = pd.concat(pd.read_json(json) for json in jsons)

ytmusic = YTMusic("oauth.json")

# auto reload library
%load_ext autoreload
%autoreload 2

In [ ]:
from tqdm import tqdm
from time import sleep
import convert

artists = pd.DataFrame(library["artists"])

artists['channelId'] = ""

max = 100
redo = []
for i, row in tqdm(artists.iterrows(), total=len(artists)):
    if row["channelId"] == "":
        try:
            artists.loc[i, 'channelId'] = convert.search_for_artist(ytmusic, row["name"])
        except:
            print(f"Error with {row['name']}")
            redo.append(i)
            
        if i % max == 0:
            tqdm.write(f"Sleeping for 15 seconds")
            sleep(5)

# artists.to_csv("artist_cids_uncorrected.csv", index=False)


In [ ]:
# artists = pd.read_csv("artist_cids_uncorrected.csv")

empty = []
fixed = []

artists['test_track'] = ""

for index, row in artists.iterrows():
    test_track = ""
    if row["name"] in full_streaming_history["master_metadata_album_artist_name"].values:
        test_track = full_streaming_history[full_streaming_history["master_metadata_album_artist_name"] == row["name"]].iloc[0]['master_metadata_track_name']
    artists.loc[index, 'test_track'] = test_track


def get_correct_id(index, row):
    try:
        return ytmusic.search(f"{row['name']} - {row['test_track']}", filter="songs", ignore_spelling=True)[0]["artists"][0]["id"]
    except:
        empty.append([index, ytmusic.search(f"{row['name']} - {row['test_track']}", filter="songs", ignore_spelling=True)])
        return row["channelId"]
    
for index, row in tqdm(artists.iterrows(), total=len(artists)):
    if row["test_track"] != "":
        correct_id = get_correct_id(index, row)
        if correct_id != row["channelId"]:
            fixed.append([
                index,
                row["name"],
                row["channelId"],
                correct_id
            ])
            artists.loc[index, 'channelId'] = correct_id

artists['alternateChannelId'] = ""

for index, artist_name, alt_id, new_id in fixed:
    if alt_id == artists.iloc[index]['channelId']:
        continue
    if new_id != artists.iloc[index]['channelId']:
        continue
    artists.loc[index, 'alternateChannelId'] = alt_id


In [ ]:
# check which artists have not been corrected
def channel_url(channel_id):
    return f"https://music.youtube.com/channel/{channel_id}"

[(k[0], f"{artists.iloc[k[0]]['name']} - {artists.iloc[k[0]]['test_track']}", channel_url(artists.iloc[k[0]]["channelId"])) for k in empty],
[(k[1], f"{channel_url(k[2])}",f"{channel_url(k[3])}") for k in fixed]

In [ ]:
# update subscribed status for artists
for channelId in tqdm(artists.channelId.values):
    if channelId == "" or pd.isna(channelId):
        continue
    artists.loc[artists['channelId'] == channelId,
                'subscribed'] = ytmusic.get_artist(channelId)['subscribed']

In [ ]:
#subscribe to artists
# needs channelId, subscribed, alternateChannelId, name for each artist

responses = []
for index, channelId in enumerate(tqdm(artists[artists['subscribed'] == False].channelId.values)):

    if channelId == "" or pd.isna(channelId) or artists.iloc[index]['subscribed'] == True:
        continue

    if ytmusic.get_artist(channelId)['subscribed']:
        continue

    responses.append([index, ytmusic.subscribe_artists(channelId)])

    sleep(2)
    if not ytmusic.get_artist(channelId)['subscribed']:
        alt_id = artists.iloc[index]['alternateChannelId']

        if alt_id != "":
            responses.append([index, ytmusic.subscribe_artists(alt_id)])

            sleep(2)
            if not ytmusic.get_artist(alt_id)['subscribed']:
                print(f"Failed to subscribe to {artists.iloc[index]['name']}"
                      )